In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
import json
import random

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomFlip("vertical"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])
def load_and_preprocess_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    #img = tf.image.rgb_to_grayscale(img)
    img = tf.image.resize(img, (512,512))
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

def make_dataset(paths, labels, augment=False):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                    num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
    return ds

samples = random.sample(annotations, 9)

paths = []
labels = []

for sample in samples:
    image = images[sample["image_id"]-1]
    #u jsonu se nalaze krivi nazivi slika pa je potrebno ispraviti
    paths.append("../fastener_dataset/images/" + image["file_name"].replace(":","_"))
    labels.append(sample["category_id"])

ds = make_dataset(paths, labels, augment=True)

plt.figure(figsize=(10, 10))
for images, labels in ds.take(1):
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy())
        plt.title(f"Label: {labels[i].numpy()}")
        plt.axis("off")

plt.suptitle("Random Augmented Training Images")
plt.show()
